In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
import pickle

In [2]:
train_joke_df = pd.read_csv(r'..\data\recsys-in-practice\train_joke_df.csv')

In [3]:
train_joke_df["UID"] = train_joke_df["UID"].astype(int)
train_joke_df["JID"] = train_joke_df["JID"].astype(int)

train_joke_df['UID'] = train_joke_df['UID'] - 1
train_joke_df['JID'] = train_joke_df['JID'] - 1

In [4]:
train_joke_df

,UID,JID,Rating
0,18028,5,-1.26
1,3297,63,-4.17
2,3365,57,0.92
3,12734,91,3.69
4,11364,37,-6.60
...,...,...,...
1448359,22603,25,2.82
1448360,22254,35,-1.94
1448361,21055,39,-9.56
1448362,12327,96,0.87


In [5]:
train_df, valid_df = train_test_split(train_joke_df, test_size=0.0007, random_state=42)

len(valid_df)

1014

In [6]:
train_df["UID"] = train_df["UID"].astype(int)
train_df["JID"] = train_df["JID"].astype(int)
valid_df["UID"] = valid_df["UID"].astype(int)
valid_df["JID"] = valid_df["JID"].astype(int)


# сделаем сортировку и перепишем index
train_df = train_df.sort_values(by=['UID', 'JID'])
train_df = train_df.reset_index(drop=True)

valid_df = valid_df.sort_values(by=['UID', 'JID'])
valid_df = valid_df.reset_index(drop=True)

In [7]:
from catboost import CatBoostRanker, Pool, MetricVisualizer

In [8]:
features = ['UID', 'JID']
cat_features = ['UID', 'JID']

In [9]:
train_pool = Pool(train_df[features], label=train_df['Rating'], group_id=train_df['UID'], cat_features=cat_features)
valid_pool = Pool(valid_df[features], label=valid_df['Rating'], group_id=valid_df['UID'], cat_features=cat_features)


In [10]:
default_parameters = {
    'iterations': 2000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE',
}

In [11]:
model = CatBoostRanker(**default_parameters)
model.fit(train_pool, eval_set=valid_pool, plot=True)

C:\ProgramData\Anaconda3\envs\torchvision2\lib\site-packages\catboost\core.py:6240: RuntimeWarning: Regression loss ('RMSE') ignores an important ranking parameter 'group_id'
  warnings.warn("Regression loss ('{}') ignores an important ranking parameter 'group_id'".format(loss_function), RuntimeWarning)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.104587
0:	learn: 5.1149050	test: 5.0327435	best: 5.0327435 (0)	total: 433ms	remaining: 14m 25s
1:	learn: 5.0158097	test: 4.9133496	best: 4.9133496 (1)	total: 673ms	remaining: 11m 12s
2:	learn: 4.9346973	test: 4.8152039	best: 4.8152039 (2)	total: 795ms	remaining: 8m 49s
3:	learn: 4.8673783	test: 4.7360597	best: 4.7360597 (3)	total: 912ms	remaining: 7m 35s
4:	learn: 4.8096012	test: 4.6663701	best: 4.6663701 (4)	total: 1.03s	remaining: 6m 51s
5:	learn: 4.7615122	test: 4.6070215	best: 4.6070215 (5)	total: 1.27s	remaining: 7m 3s
6:	learn: 4.7222274	test: 4.5567861	best: 4.5567861 (6)	total: 1.39s	remaining: 6m 36s
7:	learn: 4.6884164	test: 4.5087800	best: 4.5087800 (7)	total: 1.51s	remaining: 6m 16s
8:	learn: 4.6613761	test: 4.4707866	best: 4.4707866 (8)	total: 1.63s	remaining: 6m 1s
9:	learn: 4.6384986	test: 4.4392065	best: 4.4392065 (9)	total: 1.75s	remaining: 5m 48s
10:	learn: 4.6192990	test: 4.4128329	best: 4.4128329 (10)	total: 1.87s	remaining: 5m 37s
11:	learn: 

94:	learn: 4.5092032	test: 4.2142352	best: 4.2142352 (94)	total: 11.8s	remaining: 3m 56s
95:	learn: 4.5091695	test: 4.2140105	best: 4.2140105 (95)	total: 11.9s	remaining: 3m 56s
96:	learn: 4.5091273	test: 4.2139917	best: 4.2139917 (96)	total: 12.1s	remaining: 3m 56s
97:	learn: 4.5090739	test: 4.2141879	best: 4.2139917 (96)	total: 12.2s	remaining: 3m 56s
98:	learn: 4.5090546	test: 4.2140694	best: 4.2139917 (96)	total: 12.3s	remaining: 3m 56s
99:	learn: 4.5090538	test: 4.2140658	best: 4.2139917 (96)	total: 12.4s	remaining: 3m 55s
100:	learn: 4.5090103	test: 4.2137585	best: 4.2137585 (100)	total: 12.5s	remaining: 3m 54s
101:	learn: 4.5089766	test: 4.2137104	best: 4.2137104 (101)	total: 12.6s	remaining: 3m 54s
102:	learn: 4.5089743	test: 4.2137321	best: 4.2137104 (101)	total: 12.7s	remaining: 3m 53s
103:	learn: 4.5089669	test: 4.2137731	best: 4.2137104 (101)	total: 12.8s	remaining: 3m 53s
104:	learn: 4.5089428	test: 4.2136136	best: 4.2136136 (104)	total: 13s	remaining: 3m 53s
105:	learn: 4

186:	learn: 4.5068834	test: 4.2085764	best: 4.2085764 (186)	total: 22.9s	remaining: 3m 42s
187:	learn: 4.5068775	test: 4.2085846	best: 4.2085764 (186)	total: 23s	remaining: 3m 41s
188:	learn: 4.5068446	test: 4.2084355	best: 4.2084355 (188)	total: 23.2s	remaining: 3m 41s
189:	learn: 4.5068154	test: 4.2084773	best: 4.2084355 (188)	total: 23.3s	remaining: 3m 41s
190:	learn: 4.5068057	test: 4.2084478	best: 4.2084355 (188)	total: 23.4s	remaining: 3m 41s
191:	learn: 4.5067972	test: 4.2084551	best: 4.2084355 (188)	total: 23.5s	remaining: 3m 41s
192:	learn: 4.5067797	test: 4.2083965	best: 4.2083965 (192)	total: 23.6s	remaining: 3m 41s
193:	learn: 4.5067538	test: 4.2083983	best: 4.2083965 (192)	total: 23.8s	remaining: 3m 41s
194:	learn: 4.5067180	test: 4.2083522	best: 4.2083522 (194)	total: 23.9s	remaining: 3m 41s
195:	learn: 4.5067084	test: 4.2082594	best: 4.2082594 (195)	total: 24s	remaining: 3m 40s
196:	learn: 4.5066981	test: 4.2081920	best: 4.2081920 (196)	total: 24.1s	remaining: 3m 40s
197

...

1631:	learn: 4.5006225	test: 4.2013869	best: 4.2009260 (1460)	total: 3m 31s	remaining: 47.6s
1632:	learn: 4.5006188	test: 4.2013918	best: 4.2009260 (1460)	total: 3m 31s	remaining: 47.5s
1633:	learn: 4.5006182	test: 4.2013866	best: 4.2009260 (1460)	total: 3m 31s	remaining: 47.3s
1634:	learn: 4.5006172	test: 4.2014117	best: 4.2009260 (1460)	total: 3m 31s	remaining: 47.2s
1635:	learn: 4.5006161	test: 4.2014054	best: 4.2009260 (1460)	total: 3m 31s	remaining: 47.1s
1636:	learn: 4.5006160	test: 4.2014097	best: 4.2009260 (1460)	total: 3m 31s	remaining: 46.9s
1637:	learn: 4.5006142	test: 4.2013942	best: 4.2009260 (1460)	total: 3m 31s	remaining: 46.8s
1638:	learn: 4.5006133	test: 4.2013890	best: 4.2009260 (1460)	total: 3m 31s	remaining: 46.7s
1639:	learn: 4.5006121	test: 4.2013942	best: 4.2009260 (1460)	total: 3m 32s	remaining: 46.5s
1640:	learn: 4.5006112	test: 4.2013730	best: 4.2009260 (1460)	total: 3m 32s	remaining: 46.4s
1641:	learn: 4.5006090	test: 4.2013701	best: 4.2009260 (1460)	total: 3

1721:	learn: 4.5004670	test: 4.2012208	best: 4.2009260 (1460)	total: 3m 42s	remaining: 36s
1722:	learn: 4.5004645	test: 4.2012578	best: 4.2009260 (1460)	total: 3m 43s	remaining: 35.9s
1723:	learn: 4.5004640	test: 4.2012582	best: 4.2009260 (1460)	total: 3m 43s	remaining: 35.7s
1724:	learn: 4.5004626	test: 4.2012453	best: 4.2009260 (1460)	total: 3m 43s	remaining: 35.6s
1725:	learn: 4.5004623	test: 4.2012581	best: 4.2009260 (1460)	total: 3m 43s	remaining: 35.5s
1726:	learn: 4.5004594	test: 4.2011737	best: 4.2009260 (1460)	total: 3m 43s	remaining: 35.3s
1727:	learn: 4.5004584	test: 4.2011814	best: 4.2009260 (1460)	total: 3m 43s	remaining: 35.2s
1728:	learn: 4.5004573	test: 4.2011837	best: 4.2009260 (1460)	total: 3m 43s	remaining: 35.1s
1729:	learn: 4.5004559	test: 4.2012000	best: 4.2009260 (1460)	total: 3m 44s	remaining: 35s
1730:	learn: 4.5004506	test: 4.2013630	best: 4.2009260 (1460)	total: 3m 44s	remaining: 34.8s
1731:	learn: 4.5004490	test: 4.2013694	best: 4.2009260 (1460)	total: 3m 44

1811:	learn: 4.5003106	test: 4.2012890	best: 4.2009260 (1460)	total: 3m 55s	remaining: 24.4s
1812:	learn: 4.5003088	test: 4.2012918	best: 4.2009260 (1460)	total: 3m 55s	remaining: 24.3s
1813:	learn: 4.5003072	test: 4.2013010	best: 4.2009260 (1460)	total: 3m 55s	remaining: 24.1s
1814:	learn: 4.5003057	test: 4.2013180	best: 4.2009260 (1460)	total: 3m 55s	remaining: 24s
1815:	learn: 4.5003033	test: 4.2013456	best: 4.2009260 (1460)	total: 3m 55s	remaining: 23.9s
1816:	learn: 4.5003020	test: 4.2013424	best: 4.2009260 (1460)	total: 3m 55s	remaining: 23.7s
1817:	learn: 4.5003016	test: 4.2013399	best: 4.2009260 (1460)	total: 3m 55s	remaining: 23.6s
1818:	learn: 4.5002994	test: 4.2013486	best: 4.2009260 (1460)	total: 3m 55s	remaining: 23.5s
1819:	learn: 4.5002987	test: 4.2013500	best: 4.2009260 (1460)	total: 3m 56s	remaining: 23.4s
1820:	learn: 4.5002973	test: 4.2012692	best: 4.2009260 (1460)	total: 3m 56s	remaining: 23.2s
1821:	learn: 4.5002945	test: 4.2012402	best: 4.2009260 (1460)	total: 3m 

1901:	learn: 4.5001701	test: 4.2009538	best: 4.2009260 (1460)	total: 4m 7s	remaining: 12.7s
1902:	learn: 4.5001686	test: 4.2009234	best: 4.2009234 (1902)	total: 4m 7s	remaining: 12.6s
1903:	learn: 4.5001675	test: 4.2008958	best: 4.2008958 (1903)	total: 4m 7s	remaining: 12.5s
1904:	learn: 4.5001629	test: 4.2008820	best: 4.2008820 (1904)	total: 4m 7s	remaining: 12.4s
1905:	learn: 4.5001587	test: 4.2008710	best: 4.2008710 (1905)	total: 4m 7s	remaining: 12.2s
1906:	learn: 4.5001562	test: 4.2008912	best: 4.2008710 (1905)	total: 4m 7s	remaining: 12.1s
1907:	learn: 4.5001546	test: 4.2008868	best: 4.2008710 (1905)	total: 4m 8s	remaining: 12s
1908:	learn: 4.5001543	test: 4.2008850	best: 4.2008710 (1905)	total: 4m 8s	remaining: 11.8s
1909:	learn: 4.5001537	test: 4.2008838	best: 4.2008710 (1905)	total: 4m 8s	remaining: 11.7s
1910:	learn: 4.5001534	test: 4.2008846	best: 4.2008710 (1905)	total: 4m 8s	remaining: 11.6s
1911:	learn: 4.5001529	test: 4.2008832	best: 4.2008710 (1905)	total: 4m 8s	remaini

1991:	learn: 4.5000348	test: 4.2008632	best: 4.2006803 (1946)	total: 4m 19s	remaining: 1.04s
1992:	learn: 4.5000310	test: 4.2008992	best: 4.2006803 (1946)	total: 4m 19s	remaining: 912ms
1993:	learn: 4.5000276	test: 4.2009190	best: 4.2006803 (1946)	total: 4m 19s	remaining: 782ms
1994:	learn: 4.5000255	test: 4.2009159	best: 4.2006803 (1946)	total: 4m 19s	remaining: 652ms
1995:	learn: 4.5000250	test: 4.2009187	best: 4.2006803 (1946)	total: 4m 20s	remaining: 521ms
1996:	learn: 4.5000247	test: 4.2009092	best: 4.2006803 (1946)	total: 4m 20s	remaining: 391ms
1997:	learn: 4.5000233	test: 4.2009119	best: 4.2006803 (1946)	total: 4m 20s	remaining: 261ms
1998:	learn: 4.5000215	test: 4.2009160	best: 4.2006803 (1946)	total: 4m 20s	remaining: 130ms
1999:	learn: 4.5000198	test: 4.2008897	best: 4.2006803 (1946)	total: 4m 20s	remaining: 0us

bestTest = 4.200680346
bestIteration = 1946

Shrink model to first 1947 iterations.


In [12]:
model.save_model('models/catboost')

In [13]:
assert False

AssertionError: 

In [ ]:
n_users = np.unique(train_joke_df['UID']).size
n_items = np.unique(train_joke_df['JID']).size

test_set_all = np.zeros((n_users * n_items, 2))
for u in tqdm(range(n_users)):
    for j in range(n_items):
        test_set_all[(n_items * u) + j, 0] = u
        test_set_all[(n_items * u) + j, 1] = j

df_test = pd.DataFrame(test_set_all, columns=['UID', 'JID'])

df_test['UID'] = df_test['UID'].astype(int)
df_test['JID'] = df_test['JID'].astype(int)
df_test

In [ ]:

test_pool = Pool(df_test[features], cat_features=cat_features)


In [ ]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))


In [ ]:
predictions = model.predict(test_pool)


In [ ]:
df_test['Rating_pred'] = predictions
df_test

In [ ]:
mrg = df_test.merge(train_df, on=['UID', 'JID'], how="left", indicator=True)
mrg

In [ ]:
mrg = mrg[mrg['_merge'] == 'left_only']
mrg

In [ ]:
def get_n_recommendations_for_user(df, user_id, n, sort_by):
    recommended_items = df[df['UID'] == user_id]
    recommended_items = recommended_items.sort_values(sort_by, ascending=False)  
    return recommended_items.iloc[:n]

In [ ]:
n_recommendations = 10

In [ ]:
frames = []
for user in tqdm(range(n_users)):
    frames.append(get_n_recommendations_for_user(mrg, user, n_recommendations, 'Rating_pred'))
    
df_rec = pd.concat(frames).reset_index()
df_rec = df_rec.drop(columns=['index'])
df_rec

In [ ]:
df_rec.merge(train_df, on=['UID','JID'])

In [ ]:
df_rec.merge(valid_df, on=['UID','JID'])

In [ ]:
catboost_predictions = list(df_rec.groupby('UID').agg({'JID':list})['JID'].values)

In [ ]:
predictions_df_catboost = {'catboost':df_rec}
with open('predictions_df_catboost.pkl', 'wb') as f:
    pickle.dump(predictions_df_catboost, f)